<a href="https://colab.research.google.com/github/KenaMuigai/Machine_Learning_Assignments/blob/main/task_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

121800 Nicholas Wariua
121199 Muigai Joy

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype, is_numeric_dtype
import io
import re
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model,metrics
from sklearn.linear_model import Lasso,LassoCV
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV

1. Grab and prepare a dataset description set

In [13]:
from google.colab import files
uploaded = files.upload()

Saving modified_data.csv to modified_data (1).csv


In [14]:
df = pd.read_csv(io.BytesIO(uploaded['modified_data.csv']))
print (df)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

2. Prepare the dataset for training

In [15]:
df.shape

(1460, 77)

In [16]:
print(df)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

In [17]:
newdf = df.drop('Id', axis = 'columns', inplace = False)

In [18]:
print(newdf)

      MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0             60       RL         65.0     8450   Pave      Reg         Lvl   
1             20       RL         80.0     9600   Pave      Reg         Lvl   
2             60       RL         68.0    11250   Pave      IR1         Lvl   
3             70       RL         60.0     9550   Pave      IR1         Lvl   
4             60       RL         84.0    14260   Pave      IR1         Lvl   
...          ...      ...          ...      ...    ...      ...         ...   
1455          60       RL         62.0     7917   Pave      Reg         Lvl   
1456          20       RL         85.0    13175   Pave      Reg         Lvl   
1457          70       RL         66.0     9042   Pave      Reg         Lvl   
1458          20       RL         68.0     9717   Pave      Reg         Lvl   
1459          20       RL         75.0     9937   Pave      Reg         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPor

In [19]:
df.head(583)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,579,160,FV,34.0,3604,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,2,2008,WD,Abnorml,146000
579,580,50,RM,81.0,12150,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,11,2008,WD,Normal,131500
580,581,20,RL,NaN,14585,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,6,2007,WD,Normal,181900
581,582,20,RL,98.0,12704,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2009,New,Partial,253293


- Cleaning the dataset
```
# This is formatted as code
```



In [20]:
newdf.head(4)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000


In [21]:
newdf.shape

(1460, 76)

In [22]:
newdf.isna().any

<bound method NDFrame._add_numeric_operations.<locals>.any of       MSSubClass  MSZoning  LotFrontage  LotArea  Street  LotShape  \
0          False     False        False    False   False     False   
1          False     False        False    False   False     False   
2          False     False        False    False   False     False   
3          False     False        False    False   False     False   
4          False     False        False    False   False     False   
...          ...       ...          ...      ...     ...       ...   
1455       False     False        False    False   False     False   
1456       False     False        False    False   False     False   
1457       False     False        False    False   False     False   
1458       False     False        False    False   False     False   
1459       False     False        False    False   False     False   

      LandContour  Utilities  LotConfig  LandSlope  ...  EnclosedPorch  \
0           False      

In [23]:
original_features = list(newdf.columns)
original_features
len(original_features)

76

In [24]:
features_with_missing_values = list(newdf.columns[newdf.isna().any()])
features_with_missing_values
len(features_with_missing_values)

15

In [25]:
len(newdf)

1460

In [26]:
for feature in features_with_missing_values:
  print(feature,' ',(newdf[feature].isna().sum())/len(newdf)*100)

LotFrontage   17.73972602739726
MasVnrType   0.547945205479452
MasVnrArea   0.547945205479452
BsmtQual   2.5342465753424657
BsmtCond   2.5342465753424657
BsmtExposure   2.6027397260273974
BsmtFinType1   2.5342465753424657
BsmtFinType2   2.6027397260273974
Electrical   0.0684931506849315
FireplaceQu   47.26027397260274
GarageType   5.5479452054794525
GarageYrBlt   5.5479452054794525
GarageFinish   5.5479452054794525
GarageQual   5.5479452054794525
GarageCond   5.5479452054794525


In [27]:
df.dropna(axis='columns', thresh = 0.5*len(newdf), inplace=True)
#Remove the number of null values that exceeds half of the total records

#We can fill the missing prices with mean or median price of the entire column.
#Backfill, bfill, (Backward propagated/filled from the bottom going upwards) is the opposite of forward fill, ffill(Forward propagated/filled
# from the top going downwards)
#https://towardsdatascience.com/how-to-fill-missing-data-with-pandas-8cb875362a0d

In [28]:
mean_fill=['LotFrontage','MasVnrArea']
bfill=['FireplaceQu']
ffill=list(set(features_with_missing_values)-set(mean_fill)-set(bfill))

In [29]:
ffill

['GarageType',
 'BsmtCond',
 'BsmtFinType1',
 'MasVnrType',
 'Electrical',
 'GarageQual',
 'GarageYrBlt',
 'BsmtExposure',
 'BsmtFinType2',
 'GarageFinish',
 'GarageCond',
 'BsmtQual']

In [30]:
print(len(ffill))

12


In [31]:
#use the mean of this column to populate the empty 
for feature in mean_fill:
  df[feature].fillna(df[feature].mean(),inplace=True)

In [32]:
for feature in ffill:
  df[feature].fillna(method='ffill',inplace=True)

In [33]:
for feature in bfill:
  df[feature].fillna(method='bfill',inplace=True)

In [34]:
df.isna().any()

Id               False
MSSubClass       False
MSZoning         False
LotFrontage      False
LotArea          False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 77, dtype: bool

4. ENCODE THE DATASET

In [35]:
df.dtypes
#a form of standardisation on the datatypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 77, dtype: object

In [36]:
dog_data=df.select_dtypes(include=['object']).copy()
#select all from the dataset and show the ones with the object datatype
dog_data.head(4)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml


In [37]:
dog_features=list(dog_data.columns)
dog_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [38]:
print(len(dog_features))

39


In [39]:
#Categorise the columns
nominal=['MSZoning','LandContour','LotConfig','Neighborhood']
ordinal=list(set(dog_features)-set(nominal))
numerical=list(set(original_features)-set(dog_features))
target=['SalePrice']

In [40]:
df[target]
#What we are trying to predict 
#Y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [41]:
for feature in ordinal:
#all ordinal features into a category
  df[feature]=(df[feature].astype('category')).cat.codes
#cat codes encodes each unique entry in the colum into a specific number

In [42]:
df_nominal=pd.get_dummies(df[nominal])
#dummy variables and one hot encoding

In [43]:
df_ordinal=df[ordinal]

In [44]:
df_numerical=df[numerical]

In [45]:
encoded_df=pd.concat([df_numerical,df_nominal,df_ordinal],axis=1)
#Joining the columns

In [46]:
encoded_df.head(6)

,BedroomAbvGr,1stFlrSF,LowQualFinSF,BsmtFullBath,PoolArea,EnclosedPorch,HalfBath,2ndFlrSF,LotArea,TotRmsAbvGrd,...,SaleCondition,LotShape,KitchenQual,Functional,LandSlope,Foundation,Electrical,Heating,ExterCond,Exterior1st
0,3,856,0,1,0,0,1,854,8450,8,...,4,3,2,6,0,2,4,1,4,12
1,3,1262,0,0,0,0,0,0,9600,6,...,4,3,3,6,0,1,4,1,4,8
2,3,920,0,1,0,0,1,866,11250,6,...,4,0,2,6,0,2,4,1,4,12
3,3,961,0,1,0,272,0,756,9550,7,...,0,0,2,6,0,0,4,1,4,13
4,4,1145,0,1,0,0,1,1053,14260,9,...,4,0,2,6,0,2,4,1,4,12
5,1,796,0,1,0,0,1,566,14115,5,...,4,0,3,6,0,5,4,1,4,12


In [47]:
encoded_df.shape

(1460, 111)

#data normalization

In [48]:
encoded_df[target]

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [49]:
#dropping sale price as its the target
X=encoded_df.drop('SalePrice',axis=1)

In [50]:
y=df[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [51]:
X=X.to_numpy()
X
#convert to numpyarray

array([[3.000e+00, 8.560e+02, 0.000e+00, ..., 1.000e+00, 4.000e+00,
        1.200e+01],
       [3.000e+00, 1.262e+03, 0.000e+00, ..., 1.000e+00, 4.000e+00,
        8.000e+00],
       [3.000e+00, 9.200e+02, 0.000e+00, ..., 1.000e+00, 4.000e+00,
        1.200e+01],
       ...,
       [4.000e+00, 1.188e+03, 0.000e+00, ..., 1.000e+00, 2.000e+00,
        5.000e+00],
       [2.000e+00, 1.078e+03, 0.000e+00, ..., 1.000e+00, 4.000e+00,
        8.000e+00],
       [3.000e+00, 1.256e+03, 0.000e+00, ..., 1.000e+00, 4.000e+00,
        6.000e+00]])

In [52]:
y=y.to_numpy()
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [53]:
X.shape

(1460, 110)

In [54]:
y.shape

(1460, 1)

In [55]:
#resize distributioon where mean = 0 and standard deviation = 1
X=StandardScaler().fit_transform(X)
y=StandardScaler().fit_transform(y)

11/06/2022

In [56]:
#from all the features, combine all and give 2 if using n_components=2
pca = PCA(n_components=2)

In [59]:
p_components=pca.fit_transform(X)
#combination of fit + tranform
#fit - uses the required method - compute mean and standard deviation
#transform - apply what youve used for fit 

In [60]:
p_components

array([[ 2.40853621, -1.33013882],
       [ 0.43196006,  2.62180885],
       [ 2.82844476, -0.85513212],
       ...,
       [ 1.25988356, -0.86516157],
       [-2.83066209,  3.1822035 ],
       [-0.75664363,  2.68247099]])

In [61]:
pca.explained_variance_ratio_

array([0.10207313, 0.04345606])

Training our model

In [63]:
regressor = LassoCV()

In [66]:
regressor.fit(p_components,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [67]:
regressor.coef_

array([0.25351583, 0.01759882])

In [69]:
regressor.intercept_

4.2889648973774405e-17

Data Splitting

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [72]:
y_train

array([[ 1.52462833],
       [-0.69786554],
       [ 0.65791868],
       ...,
       [-0.3767687 ],
       [ 0.99576293],
       [-0.60972131]])

MODEL SELECTION

In [73]:
result = {}
models = {'LinearRegression':linear_model.LinearRegression(),
          'Lasso':linear_model.Lasso(),
          'Ridge':linear_model.Ridge(),
          'ElasticNet':linear_model.ElasticNet()}

for i in models:
  models[i].fit(X_train,y_train)
  accuracy = models[i].score(X_test,y_test)
  result[i]=[accuracy]
pd.DataFrame(result)

,LinearRegression,Lasso,Ridge,ElasticNet
0,0.749933,-0.000834,0.749666,0.328192


K-Fold Cross Validation

In [74]:
kfold = KFold(n_splits=4)
for i in models:
  regressor = models[i]
  c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
  accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

,LinearRegression,Lasso,Ridge,ElasticNet
0,-1.210928e+20,-0.005458,0.850559,0.332694


Hyperparameter Tuning using Grid Search

In [75]:
for i in models:
  regressor = models[i]
  if i == 'Lasso':
    lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03],'max_iter':[1000,1100,2000],'tol':[0.001,0.0001,0.000001]}
    grid = GridSearchCV(regressor,param_grid=lasso_params,cv=2,n_jobs=-1)
    results = grid.fit(X_train,y_train)
    print("Best Params are ", results.best_params_)
    #accuracy = results.score(X_test,y_test)
    accuracy=results.best_score_
  else:    
    c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
    accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

Best Params are  {'alpha': 0.02, 'max_iter': 1000, 'tol': 0.001}


,LinearRegression,Lasso,Ridge,ElasticNet
0,-1.210928e+20,0.857301,0.850559,0.332694


Hyperparameter Tuning using Random Search

In [76]:
for i in models:
  regressor = models[i]
  if i == 'Lasso':
    random = RandomizedSearchCV(regressor,param_distributions=lasso_params,cv=3,n_jobs=-1)
    results = random.fit(X_train,y_train)
    print(results.best_score_)
    print("Best Params are ", results.best_params_)
    #accuracy = results.score(X_test,y_test)
    accuracy=results.best_score_
  else:    
    c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
    accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

0.8580158581346379
Best Params are  {'tol': 0.0001, 'max_iter': 1100, 'alpha': 0.02}


,LinearRegression,Lasso,Ridge,ElasticNet
0,-1.210928e+20,0.858016,0.850559,0.332694
